In [2]:
import keras, os, pickle, re, sklearn, string, tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.layers import Embedding
from keras.optimizers import Adadelta, adam
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils.vis_utils  import plot_model
from sklearn import preprocessing
from keras import optimizers

print('Keras version: \t\t%s' % keras.__version__)
print('Scikit version: \t%s' % sklearn.__version__)
print('TensorFlow version: \t%s' % tensorflow.__version__)


Using TensorFlow backend.


Keras version: 		2.0.9
Scikit version: 	0.19.1
TensorFlow version: 	1.4.1


In [3]:
import nltk
from nltk.corpus import stopwords
from tika import parser
from sklearn.metrics import classification_report



In [4]:
def LoadData_level5():
    label_level5=[]
    i=0
    for foldername in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level5/developer"):
        for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level5/developer/"+foldername):
    
            try:
                print (i, foldername, file)
                parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level5/developer/"+foldername+"/"+file)

                resume_contents=clean_doc(parsedPDF['content'])
            #Data = resume_contents.encode('utf-8')    
                label_level5.append((resume_contents,foldername))
            except UnicodeEncodeError:
                print ('Unicode error:', file)
            i=i+1
    #print (label_resume)
    return(label_level5)
        

In [4]:
#Preprocessing
def clean_doc(doc):
    """
    Cleaning a document by several methods:
        - Lowercase
        - Removing whitespaces
        - Removing stopwords
        - Removing punctuations
        
    """
    stop_words = set(stopwords.words('english'))
    
    # Lowercase
    doc = doc.lower()
   
    tokens = doc.translate({ord(c):"" for c in "\u200b\uf020\u2028\xa0\uf0e0\uf095\uf041\uf0e1\uf0b7\xad"})
    tokens = tokens.translate({ord(c):" " for c in "[):,·](;•●■♦▪"})
    tokens = tokens.translate({ord(c):"f" for c in "�"})
    
    # Removing multiple whitespaces
    tokens = re.sub(r"\?", " \? ", tokens)
    
    # Split in tokens
    tokens = tokens.split()
    
    # Remove Stopwords
    tokens = [w for w in tokens if not w in stop_words]
    # Remove punctuation
    tokens = [w.translate(str.maketrans('', '', string.punctuation)) for w in tokens]
    
    return ' '.join(tokens)

In [6]:
label_level5=LoadData_level5()

0 php reshmakr.pdf
1 php MAYAARJUNAN[1_0].docx
2 php arifca[5_1].pdf
3 php BinsBabu.pdf
4 php priyankaphilip[1_4].pdf
5 php VanithaB[1_9].docx
6 php seshasaikotapati[7_0].doc
7 php vijay_koogu_Bengaluru___Bangalore_7.03_yrs.docx
8 php Anju K N.docx
9 php pankaj_shukla_Nagpur_10.01_yrs.doc
10 php NaheedaKamarudheen_Resume.pdf
11 php Ranjit_Kumar_Delhi_Region_7.08_yrs.doc
12 php Sheethal PHP,Codeigniter,Wordpress.doc
13 php Anand_Arunachalam_Chennai_5.06_yrs.doc
14 php rajesh_vishwakarma_Delhi_Region_8.00_yrs.doc
15 php panchalkeval[1_10].pdf
16 php ASHIK cV.pdf
17 php saranya_v_Chennai_6.03_yrs.doc
18 php Ajay_kumar_singh__Bengaluru___Bangalore_12.00_yrs.doc
19 php MohdSamiullah[3_6].doc
20 php sajeevsomanathan[2_11].docx
21 php Sachin_Talan_Delhi_10.00_yrs.pdf
22 php SAINADHKOTINADHAM[1_6].docx
23 php VijayabanuS[1_0].pdf
24 php 84632746_Gurgaon_2.05_yrs.pdf
25 php DONAJOHNN[3_1].docx
26 php sneha_harihar_Bengaluru___Bangalore_2.07_yrs.pdf
27 php ASWATHYVIJAYAN[4_0].pdf
28 php TonyThom

220 dotnet Safuvan_Paravakkal_India_4.02_yrs.docx
221 dotnet Prajnashree_Das_Bengaluru___Bangalore_3.06_yrs.docx
222 dotnet Pawan_Upadhye__Pune_3.08_yrs.pdf
223 dotnet Soumya_Regina_E_J_Cochin___Kochi___Ernakulam_4.00_yrs.doc
224 dotnet Sarath_Shanmughan_Cochin___Kochi___Ernakulam_2.08_yrs.docx
225 dotnet Pramit_Saha_Pune_3.03_yrs.pdf
226 dotnet Raji__Chennai_3.00_yrs.docx
227 dotnet pranav_joseph_Cochin___Kochi___Ernakulam_3.05_yrs.docx
228 dotnet pradeep_kumar_Hyderabad___Secunderabad_3.07_yrs.doc
229 dotnet Elangovan_Dhanabalan_Bengaluru___Bangalore_4.03_yrs (1).docx
230 dotnet Suman_S_S__Bengaluru___Bangalore_4.02_yrs.doc
231 dotnet resume_sujith.pdf
232 dotnet Sarath_Raj__Cochin___Kochi___Ernakulam_4.00_yrs.docx
233 dotnet Vipin K V_Resume_Latest - Reject.doc
234 dotnet Vinayak_Fanase_Pune_4.05_yrs.docx
235 dotnet VINOD.S.doc
236 dotnet Jagdish_Nair_Cochin___Kochi___Ernakulam_4.11_yrs.docx
237 dotnet Sparsh_jain__Gurgaon_3.00_yrs.pdf
238 dotnet Partha_Pratim_Ghosh_Bengaluru___Bang

394 pythondjango A.KRISHNA_REDDY_Hyderabad_-_Secunderabad_7.00_yrs.pdf
395 pythondjango ArunK[4_3].pdf
396 pythondjango AkhilKA[2_9].pdf
397 pythondjango ashish_anand_Chandigarh_6.11_yrs.docx
398 pythondjango HaridasanPC[6_0].docx
399 pythondjango Soumyaranjan[2_5].docx
400 pythondjango Gauri_kulkarni_Pune_1.07_yrs.pdf
401 pythondjango GaneshManturi[4_4].docx
402 pythondjango Mahesh_N_Hyderabad___Secunderabad_2.02_yrs.doc
403 pythondjango harikesh_patel_Bengaluru_-_Bangalore_6.02_yrs.docx
404 pythondjango JithinPrakash[2_4].pdf
405 pythondjango Mohan[4_1].docx
406 pythondjango shibiraj__resume.pdf
407 pythondjango Bhagyashri_Gite_Pune_2.00_yrs.docx
408 pythondjango NIKHIL KRISHNAN.doc
409 pythondjango Ottigunta_Jyothi_Naidu_India_4.02_yrs.docx
410 pythondjango VimalKesavan[3_11].docx
411 pythondjango SVINOTHKUMAR[4_8].docx
412 pythondjango veenaaravindc[2_5].docx
413 pythondjango Vineet_kumar_Dubey_Mumbai_4.10_yrs.pdf
414 pythondjango SarathKumarJ[3_7].pdf
415 pythondjango SELVAKUMAR.L

610 ios Rafeek Resume doc docx.pdf
611 ios Ramu[4_6].doc
612 ios Ambili B Menon.doc
613 ios AbinBaby.pdf
614 ios KarthikP[2_7].docx
615 ios Ajaykumar_Mishra_Nashik_2.04_yrs.docx
616 ios UNNIKRISHNANP[7_0].docx
617 ios RoshnaDsouza[5_0].pdf
618 ios SarathRaveendran[3_0].pdf
619 ios 55778200_Bengaluru___Bangalore_2.01_yrs.pdf
620 ios Ebin Dominic_iOS.docx
621 ios Britto Thomas.docx
622 ios RINCE[5_6].doc
623 ios harishthadandla[2_0].docx
624 ios BIJU GEORGE- Reject.doc
625 ios MohamedRafiBA[4_9].doc
626 ios srinu[3_2].doc
627 ios SaravananM[1_3].docx
628 ios balaram_somineni_Hyderabad___Secunderabad_2.00_yrs.docx
629 ios ImranMohammed - Reject.doc
630 ios JamalAhamad[2_3].doc
631 ios NeerajKumarMishra[3_0].doc
632 ios vamshi[6_0].doc
633 ios 72290504_Noida_1.00_yrs.doc
634 ios SANTHOSHKUMAR[3_0].doc
635 ios BollaSrinivas[2_3].pdf
636 ios Ashokkumar_Subramanian_Chennai_0.06_yrs.docx
637 ios DONIYAJOY[3_0].pdf
638 ios Ami_Intwala__Ahmedabad_1.05_yrs.pdf
639 ios AkhilaTVellanikaran[2_0].doc

824 corejava ishita__Mumbai_2.00_yrs.pdf
825 corejava Nikita_Garg_Pune_1.10_yrs.doc
826 corejava SHIBU BABU.doc
827 corejava AnnlatestResume.pdf
828 corejava Jitesh_Kumar_Bengaluru___Bangalore_0.00_yrs.doc
829 corejava Jyothsna_ranga_Hyderabad___Secunderabad_0.00_yrs.docx
830 corejava Nagaraju_Kakumanu_Bengaluru___Bangalore_0.00_yrs.doc
831 corejava rakesh_meshram_Mumbai_9.00_yrs.doc
832 corejava Junaidh M K.doc
833 corejava Harsha_Vardhana_Gangavarapu_Chennai_6.09_yrs.pdf
834 corejava Madhukar_Reddy_Gogula__Hyderabad___Secunderabad_0.00_yrs.doc
835 corejava Lakshmi_P_Hyderabad___Secunderabad_4.04_yrs.docx
836 corejava reeti_metre_Bengaluru___Bangalore_2.08_yrs.docx
837 corejava THOMAS P M.doc
838 corejava Hariprasath_Govindarajulu_Chennai_0.00_yrs.pdf
839 corejava peerambhi_dudekula_Hyderabad___Secunderabad_3.02_yrs.doc
840 corejava kundan_kumar_Gurgaon_3.00_yrs.docx
841 corejava Niladri_Sarkar__Kolkata_10.00_yrs.docx
842 corejava CodechefVaibhav_Kashyap_Delhi_5.06_yrs.docx
843 coreja

In [7]:
len(label_level5)

900

In [8]:

jdres_train_labels=[]
jdres_train_data=[]

for row in label_level5:
    jdres_train_data.append(row[0])
    jdres_train_labels.append(row[1])
#print(jdres_train_data)
#print(len(jdres_train_data))

In [9]:
print(len(jdres_train_data))
print(len(jdres_train_labels))

900
900


In [10]:
df1 = pd.DataFrame(jdres_train_data,columns=['filecontent'])
df2 = pd.DataFrame(jdres_train_labels,columns=['label'])

In [11]:
#print (df2)

In [11]:
#print(jdres_train_data[42])

In [12]:
encoder = preprocessing.LabelEncoder()
encoder.fit(jdres_train_labels)
encoded_labels = encoder.transform(jdres_train_labels)
# convert integers to dummy variables (i.e. one hot encoded)
labels = to_categorical(encoded_labels,num_classes=6)
print(encoded_labels)


[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [13]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(jdres_train_data)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(jdres_train_data)
vocab_size

39692

In [14]:
# pad documents to a max length of1000 words
max_length = 1000
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6349  3857    92 ...,     0     0     0]
 [ 8671  1489   956 ...,     0     0     0]
 [16980 16981   625 ...,     0     0     0]
 ..., 
 [ 5110   622  7085 ...,     0     0     0]
 [12752   383  3342 ...,     0     0     0]
 [11177   622 16907 ...,     0     0     0]]


In [15]:
print('Shape of data tensor:', padded_docs.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (900, 1000)
Shape of label tensor: (900, 6)


In [16]:
indices = np.arange(padded_docs.shape[0])
np.random.shuffle(indices)
padded_docs = padded_docs[indices]
labels = labels[indices]

In [17]:
#loading glove

embeddings_index = dict()
f = open('/home/shabna/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [18]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [19]:
#create model

from keras.layers import Activation, Input, Dense, Flatten, Dropout, Embedding
from keras.layers import Conv1D, MaxPooling1D,GlobalMaxPooling1D
from keras.models import Model

In [20]:
#from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [453]:
def create_model():
    model = Sequential()
    e = Embedding(vocab_size,100, weights=[embedding_matrix], input_length=1000, trainable='true')
    model.add(e)
    model.add(Conv1D(128,5, activation='relu',name='l1'))
    model.add(MaxPooling1D(pool_size=5,name='l2'))
    model.add(Conv1D(128,5, activation='relu',name='l7'))
    model.add(MaxPooling1D(pool_size=5,name='l8'))
    model.add(Conv1D(128,5, activation='relu',name='l3'))
    model.add(GlobalMaxPooling1D(name='l4'))
   
    model.add(Dense(18, activation='sigmoid',name='l5'))
    model.add(Dropout(0.2))
    #model.add(Flatten())
    model.add(Dense(6, activation='softmax',name='l6'))
    #adam = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model

In [454]:
#model = Model(embedding_matrix, preds)
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1000, 100)         3969200   
_________________________________________________________________
l1 (Conv1D)                  (None, 996, 128)          64128     
_________________________________________________________________
l2 (MaxPooling1D)            (None, 199, 128)          0         
_________________________________________________________________
l7 (Conv1D)                  (None, 195, 128)          82048     
_________________________________________________________________
l8 (MaxPooling1D)            (None, 39, 128)           0         
_________________________________________________________________
l3 (Conv1D)                  (None, 35, 128)           82048     
_________________________________________________________________
l4 (GlobalMaxPooling1D)      (None, 128)               0         
__________

In [ ]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

param_grid = dict(learn_rate=learn_rate, momentum=momentum, batch_size=batch_size, epochs=epochs,optimizer=optimizer)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
    grid_result = grid.fit(X_train, y_train,validation_data=(X_val,y_val))
    
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [456]:
from keras.callbacks import ModelCheckpoint

In [457]:
from keras.callbacks import Callback

In [458]:
filepath="/home/shabna/Desktop/example_codes/weights_5.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only="True", mode="max")
callbacks_list = [checkpoint]

In [459]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)

XX_train, X_test, yy_train, y_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=42)

i=1
for train_index, test_index in kf.split(XX_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    print(i,":num of fold")

    X_train, X_val = XX_train[train_index], XX_train[test_index]
    y_train, y_val = yy_train[train_index], yy_train[test_index]
    
    history = model.fit(X_train, y_train, validation_data=(X_val,y_val) ,epochs=50, batch_size=100, verbose=1, callbacks=callbacks_list)
   
    i=i+1
    

1 :num of fold
Train on 480 samples, validate on 240 samples
Epoch 1/50
480/480 [==============================] - 28s 58ms/step - loss: 1.8446 - acc: 0.1479 - val_loss: 1.7885 - val_acc: 0.1583
Epoch 2/50
480/480 [==============================] - 22s 47ms/step - loss: 1.7887 - acc: 0.2000 - val_loss: 1.7800 - val_acc: 0.2042
Epoch 3/50
480/480 [==============================] - 22s 45ms/step - loss: 1.7415 - acc: 0.2437 - val_loss: 1.7653 - val_acc: 0.2083
Epoch 4/50
480/480 [==============================] - 21s 45ms/step - loss: 1.7510 - acc: 0.2417 - val_loss: 1.7374 - val_acc: 0.3083
Epoch 5/50
480/480 [==============================] - 20s 42ms/step - loss: 1.6949 - acc: 0.3563 - val_loss: 1.7091 - val_acc: 0.3000
Epoch 6/50
480/480 [==============================] - 20s 43ms/step - loss: 1.6438 - acc: 0.3500 - val_loss: 1.6713 - val_acc: 0.3417
Epoch 7/50
480/480 [==============================] - 20s 43ms/step - loss: 1.5708 - acc: 0.4167 - val_loss: 1.6204 - val_acc: 0.4167
E

Epoch 6/50
480/480 [==============================] - 22s 45ms/step - loss: 0.2192 - acc: 0.9958 - val_loss: 0.1589 - val_acc: 1.0000
Epoch 7/50
480/480 [==============================] - 20s 42ms/step - loss: 0.2151 - acc: 1.0000 - val_loss: 0.1538 - val_acc: 1.0000
Epoch 8/50
480/480 [==============================] - 20s 42ms/step - loss: 0.2051 - acc: 0.9938 - val_loss: 0.1533 - val_acc: 1.0000
Epoch 9/50
480/480 [==============================] - 20s 42ms/step - loss: 0.2101 - acc: 0.9958 - val_loss: 0.1457 - val_acc: 1.0000
Epoch 10/50
480/480 [==============================] - 21s 44ms/step - loss: 0.2004 - acc: 0.9979 - val_loss: 0.1430 - val_acc: 1.0000
Epoch 11/50
480/480 [==============================] - 21s 44ms/step - loss: 0.1932 - acc: 1.0000 - val_loss: 0.1395 - val_acc: 1.0000
Epoch 12/50
480/480 [==============================] - 21s 43ms/step - loss: 0.1900 - acc: 0.9979 - val_loss: 0.1363 - val_acc: 1.0000
Epoch 13/50
480/480 [==============================] - 20s 

KeyboardInterrupt: 

In [460]:
model=load_model("weights_5.best.hdf5")


In [461]:
plot_model(model, to_file = 'models5_1.png')

In [30]:
y_train

array([[ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.]])

In [48]:
from sklearn.metrics import classification_report, confusion_matrix

In [49]:
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred,axis=1)

In [50]:
y_pred = model.predict_classes(X_test)

180/180 [==============================] - 2s 12ms/step


In [51]:
p = model.predict_proba(X_test)

180/180 [==============================] - 2s 12ms/step


In [52]:
model.evaluate(X_test,y_test)

180/180 [==============================] - 2s 13ms/step


[0.33891636663013036, 0.94999999867545237]

In [53]:
model.evaluate(X_train,y_train)

480/480 [==============================] - 6s 12ms/step


[0.22762171228726705, 0.98750000000000004]

In [55]:
labels_index = {'Android':0 ,'Corejava':1,'Dotnet':2, 'ios':3, 'php':4, 'pythondjango':5}
target_name = [t for t in labels_index.keys()]

In [56]:
print(classification_report(np.argmax(y_test,axis=1),y_pred, target_names=target_name))

              precision    recall  f1-score   support

     Android       0.92      0.96      0.94        24
    Corejava       0.95      1.00      0.97        35
      Dotnet       0.95      0.91      0.93        22
         ios       0.92      0.89      0.91        38
         php       1.00      0.94      0.97        32
pythondjango       0.97      1.00      0.98        29

 avg / total       0.95      0.95      0.95       180



In [57]:
print(confusion_matrix(np.argmax(y_test,axis=1),y_pred))

[[23  0  0  1  0  0]
 [ 0 35  0  0  0  0]
 [ 1  1 20  0  0  0]
 [ 1  1  1 34  0  1]
 [ 0  0  0  2 30  0]
 [ 0  0  0  0  0 29]]


In [58]:
from numpy import argmax
for i, item in enumerate (X_test):
    item= item.reshape(1, 1000)
    #print (item.shape)
    classes=model.predict(item)
    p=(argmax(classes),argmax(y_test[i]))
    #print(p)
    if (argmax(classes)!=argmax(y_test[i])):
        print(p)


(3, 0)
(1, 3)
(0, 3)
(3, 4)
(1, 2)
(5, 3)
(2, 3)
(0, 2)
(3, 4)


In [140]:
X_test.shape


(90, 1000)

In [141]:
X_train.shape

(288, 1000)

In [142]:
print(X_test)

[[  37   22 5612 ...,    0    0    0]
 [  37   22 4767 ...,    0    0    0]
 [  37   22 6358 ...,    0    0    0]
 ..., 
 [ 656 9558 1924 ...,    0    0    0]
 [5450 6872   20 ...,    0    0    0]
 [  37   22 5952 ...,    0    0    0]]


In [5]:
model1=load_model("weights_5_1.best.hdf5")

In [7]:
#predictions

parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level5/developer/android/Resume_Vandana Surendranath.pdf")
#parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level5/developer/corejava/Prabhath K V.docx")

contents=[clean_doc(parsedPDF['content'])]
            
print(contents)

['vandana surendranath android developer  25 years experience mobile 9746757707 email vandanasurendranathgmailcom objective work organization provides opportunities utilize knowledge skills best possible way professional growth professional summary good knowledge android sdk apis experience integrating third party libraries effective working independently teams strong debugging problem solving skills quick learner zeal learn new technologies technical skills platform windows linux mac languages java c assembly xml objective c ide eclipse android studio xcode web service json xml database sqlite professional experience currently working jr software engineer mobatia technologies technopark three months experience qburst technologies infopark koratty android developer trainee apprenticeship keltron component complex kannur  projects project name hc4u project description healthcare4u membership program empower users take charge health app easy collect keep access health information time pl

In [8]:
def prpInput(pp):
    tx = Tokenizer()
    tx.fit_on_texts(pp)
    vocab_size = len(tx.word_index)+1
    encoded_doc = tx.texts_to_sequences(pp)
    max_length = 1000
    padded_docs = pad_sequences(encoded_doc, maxlen=max_length, padding='post')
    return padded_docs

In [9]:
mat=prpInput(contents)


In [10]:
prd = model1.predict(mat)


In [12]:
labels_index = {'Android':0 ,'Corejava':1,'Dotnet':2, 'ios':3, 'php':4, 'pythondjango':5}
rev_lable_index = {}
for key in labels_index:
    rev_lable_index[labels_index[key]] = key
print(rev_lable_index)
def result(prd,contents):
    y_classes = prd.argmax(axis=-1)
    print(len(y_classes))
    lx=[]
    #rev_lable_index = {0:'Developer' , 1: 'Other', 2: 'Tester'}
    for idx,lb in enumerate(y_classes):
        lx.append([contents[idx],rev_lable_index[lb]])
    return lx

{0: 'Android', 1: 'Corejava', 2: 'Dotnet', 3: 'ios', 4: 'php', 5: 'pythondjango'}


In [13]:
result(prd,contents)

1


[['vandana surendranath android developer  25 years experience mobile 9746757707 email vandanasurendranathgmailcom objective work organization provides opportunities utilize knowledge skills best possible way professional growth professional summary good knowledge android sdk apis experience integrating third party libraries effective working independently teams strong debugging problem solving skills quick learner zeal learn new technologies technical skills platform windows linux mac languages java c assembly xml objective c ide eclipse android studio xcode web service json xml database sqlite professional experience currently working jr software engineer mobatia technologies technopark three months experience qburst technologies infopark koratty android developer trainee apprenticeship keltron component complex kannur  projects project name hc4u project description healthcare4u membership program empower users take charge health app easy collect keep access health information time p

In [14]:

pd.DataFrame(result(prd,contents),columns=['File Content','Label'])

1


,File Content,Label
0,vandana surendranath android developer 25 yea...,Android


In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()